## Spark Performance Tuning by Afaque Ahmad

Link: https://youtube.com/playlist?list=PLWAuYt0wgRcLCtWzUxNg4BjnYlCZNEVth&si=_Uldzm2HGZeIUX91  
Github: https://github.com/afaqueahmad7117/spark-experiments